In [5]:
# 工具
# 深度学习：keras
# 传统机器学习：sklearn
# 参与比较的机器学习方法

# CNN 
# LSTM 
# 朴素贝叶斯
# KNN
# SVM
# Logisticre Gression

In [6]:
# !pip install tensorflow==2.2
# !pip install numpy
# !pip install gensim

# print(np.__version__)
# print(tensorflow.__version__)

In [3]:
def load_text(train_texts,train_labels,test_texts,test_labels):
    #coding:utf-8
    print ('*load texts:')
    train_texts = open(train_texts,encoding='UTF-8').read().split('\n')
    train_labels = open(train_labels,encoding='UTF-8').read().split('\n')
    test_texts = open(test_texts,encoding='UTF-8').read().split('\n')
    test_labels = open(test_labels,encoding='UTF-8').read().split('\n')
    all_texts = train_texts + test_texts
    all_labels = train_labels + test_labels
    return all_texts,all_labels,train_texts,train_labels,test_texts,test_labels
    
def creat_tokenizer(all_texts):
    print ('*tokenizer:')
    from tensorflow.keras.preprocessing.text import Tokenizer
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    from tensorflow.keras.utils import to_categorical
    import numpy as np

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(all_texts)
    sequences = tokenizer.texts_to_sequences(all_texts)
    word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(word_index))
    data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    labels = to_categorical(np.asarray(all_labels))
    print('Shape of data tensor:', data.shape)
    print('Shape of label tensor:', labels.shape)
    return data,labels,word_index

In [4]:
import tensorflow.keras
import keras

MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 400
VALIDATION_SPLIT = 0.16
TEST_SPLIT = 0.2

all_texts,all_labels,train_texts,train_labels,test_texts,test_labels=load_text('text.txt','categroy.txt','new_text.txt','ner_cate.txt')

data,labels,word_index=creat_tokenizer(all_texts)

print ('*split data set:')
# split the data into training set, validation set, and test set
p1 = int(len(data)*(1-VALIDATION_SPLIT-TEST_SPLIT))
p2 = int(len(data)*(1-TEST_SPLIT))
x_train = data[:p1]
y_train = labels[:p1]
x_val = data[p1:p2]
y_val = labels[p1:p2]
x_test = data[p2:]
y_test = labels[p2:]
print ('train docs: '+str(len(x_train)))
print ('val docs: '+str(len(x_val)))
print ('test docs: '+str(len(x_test)))

Using TensorFlow backend.


*load texts:
*tokenizer:
Found 206233 unique tokens.
Shape of data tensor: (14104, 100)
Shape of label tensor: (14104, 14)
*split data set:
train docs: 9026
val docs: 2257
test docs: 2821


In [22]:
def CNN_define(embedding_layer,embedding_dim,max_sequence_length):
    #define for CNN
    print ('*define model CNN:')
    from keras.layers import Dense, Input, Flatten, Dropout
    from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalMaxPooling1D
    from keras.models import Sequential
    from keras.utils import plot_model

    model = Sequential()
    if embedding_layer==None:  
        model.add(Embedding(len(word_index) + 1, embedding_dim, input_length=max_sequence_length))
    else:
        model.add(embedding_layer)
    model.add(Dropout(0.2))
    model.add(Conv1D(250, 3, padding='valid', activation='relu', strides=1))
    model.add(MaxPooling1D(3))
    model.add(Flatten())
    model.add(Dense(embedding_dim, activation='relu'))
    model.add(Dense(labels.shape[1], activation='softmax'))
    model.summary()
    # plot_model(model, to_file='model.png',show_shapes=True)

    model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['acc'])
    print (model.metrics_names)
    return model

def LSTM_define(embedding_layer,embedding_dim,max_sequence_length):
    #trainning for lstm
    print ('*define model lstm：')
    from keras.layers import Dense, Input, Flatten, Dropout
    from keras.layers import LSTM, Embedding
    from keras.models import Sequential

    model = Sequential()
    if embedding_layer==None:
        model.add(Embedding(len(word_index) + 1, embedding_dim, input_length=max_sequence_length))
    else:
        model.add(embedding_layer)
    model.add(LSTM(200, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dropout(0.2))
    model.add(Dense(labels.shape[1], activation='softmax'))
    model.summary()

    model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['acc'])
    print (model.metrics_names)
    return model
    
def model_train(model_name,model):
    model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=2, batch_size=128)
    model.save(model_name)

    print ('*testing model:')
    print (model.evaluate(x_test, y_test))

In [23]:
model=CNN_define(None,EMBEDDING_DIM,MAX_SEQUENCE_LENGTH)
model_train('CNN.h5',model)

*define model CNN:
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 100, 400)          82493600  
_________________________________________________________________
dropout_6 (Dropout)          (None, 100, 400)          0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 98, 250)           300250    
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 32, 250)           0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 8000)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 400)               3200400   
_________________________________________________________________
dense_12 (Dense)             (None,

In [24]:
model=LSTM_define(None,EMBEDDING_DIM,MAX_SEQUENCE_LENGTH)
model_train('LSTM.h5',model)

*define model lstm：
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 100, 400)          82493600  
_________________________________________________________________
lstm_1 (LSTM)                (None, 200)               480800    
_________________________________________________________________
dropout_7 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 14)                2814      
Total params: 82,977,214
Trainable params: 82,977,214
Non-trainable params: 0
_________________________________________________________________
['loss', 'acc']
Train on 9026 samples, validate on 2257 samples
Epoch 1/2
9026/9026 [==============================] - 67s 7ms/step - loss: 2.2828 - acc: 0.2314 - val_loss: 2.2457 - val_acc: 0.1573
Epoch 2/2
9026/9026 [=

In [16]:
print ('(4) load word2vec as embedding...')
import gensim
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from keras.utils import plot_model
import numpy as np


w2v_model = gensim.models.KeyedVectors.load_word2vec_format('wiki.zh.text.vector', binary=False)
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
not_in_model = 0
in_model = 0
for word, i in word_index.items(): 
    if str(word) in w2v_model:
        in_model += 1
        embedding_matrix[i] = np.asarray(w2v_model[str(word)], dtype='float32')
    else:
        not_in_model += 1
print (str(not_in_model)+' words not in w2v model')
from keras.layers import Embedding
embedding_layer = Embedding(len(word_index) + 1,EMBEDDING_DIM,weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,trainable=False)

(4) load word2vec as embedding...
59406 words not in w2v model


In [25]:
model=CNN_define(embedding_layer,EMBEDDING_DIM,MAX_SEQUENCE_LENGTH)
model_train('CNN_word2vec.h5',model)

*define model CNN:
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 400)          82493600  
_________________________________________________________________
dropout_8 (Dropout)          (None, 100, 400)          0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 98, 250)           300250    
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 32, 250)           0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 8000)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 400)               3200400   
_________________________________________________________________
dense_15 (Dense)             (None,

In [26]:
model=LSTM_define(embedding_layer,EMBEDDING_DIM,MAX_SEQUENCE_LENGTH)
model_train('LSTM_word2vec.h5',model)

*define model lstm：
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 400)          82493600  
_________________________________________________________________
lstm_2 (LSTM)                (None, 200)               480800    
_________________________________________________________________
dropout_9 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 14)                2814      
Total params: 82,977,214
Trainable params: 483,614
Non-trainable params: 82,493,600
_________________________________________________________________
['loss', 'acc']
Train on 9026 samples, validate on 2257 samples
Epoch 1/2
9026/9026 [==============================] - 20s 2ms/step - loss: 1.7624 - acc: 0.4388 - val_loss: 1.0194 - val_acc: 0.6690
Epoch 2/2
9026/9

In [27]:
def tfid(all_texts):
    print ('*doc to var:')
    from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer   
    count_v0= CountVectorizer();  
    counts_all = count_v0.fit_transform(all_texts);
    count_v1= CountVectorizer(vocabulary=count_v0.vocabulary_);  
    counts_train = count_v1.fit_transform(train_texts);   
    print ("the shape of train is "+repr(counts_train.shape))  
    count_v2 = CountVectorizer(vocabulary=count_v0.vocabulary_);  
    counts_test = count_v2.fit_transform(test_texts);  
    print ("the shape of test is "+repr(counts_test.shape))

    tfidftransformer = TfidfTransformer();    
    train_data = tfidftransformer.fit(counts_train).transform(counts_train);
    test_data = tfidftransformer.fit(counts_test).transform(counts_test); 

    x_train = train_data
    y_train = train_labels
    x_test = test_data
    y_test = test_labels
    return x_train,y_train,x_test,y_test

In [28]:
x_train,y_train,x_test,y_test=tfid(all_texts)

*doc to var:
the shape of train is (5590, 201033)
the shape of test is (8514, 201033)


In [25]:
print ('*KNN:')
from sklearn.neighbors import KNeighborsClassifier  

for x in range(1,15):  
    knnclf = KNeighborsClassifier(n_neighbors=x)
    knnclf.fit(x_train,y_train)  
    preds = knnclf.predict(x_test)
    num = 0
    preds = preds.tolist()
    for i,pred in enumerate(preds):
        if int(pred) == int(y_test[i]):
            num += 1
    print ('K= '+str(x)+', precision_score:' + str(float(num) / len(preds)))


*KNN:


E:\Anaconda\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


K= 1, precision_score:0.8202959830866807


E:\Anaconda\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


K= 2, precision_score:0.7389006342494715


...

E:\Anaconda\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


K= 12, precision_score:0.7932816537467701


E:\Anaconda\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


K= 13, precision_score:0.7928118393234672
K= 14, precision_score:0.7933991073525958


E:\Anaconda\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [112]:
print ('*SVM：')
from sklearn.svm import SVC   
svclf = SVC(kernel = 'linear') 
svclf.fit(x_train,y_train)  
preds = svclf.predict(x_test)
num = 0
preds = preds.tolist()
for i,pred in enumerate(preds):
    if int(pred) == int(y_test[i]):
        num += 1
print ('precision_score:' + str(float(num) / len(preds)))


*SVM：
precision_score:0.9837209302325581


In [26]:
print ('*Naive Bayes：')
from sklearn.naive_bayes import MultinomialNB  
from sklearn import metrics
clf = MultinomialNB(alpha = 0.01)   
clf.fit(x_train, y_train);  
preds = clf.predict(x_test)
num = 0
preds = preds.tolist()
for i,pred in enumerate(preds):
    if int(pred) == int(y_test[i]):
        num += 1
print ('precision_score:' + str(float(num) / len(preds)))


*Naive Bayes：
precision_score:0.9012215175005873


In [27]:
print ('*Logisticre Gression：')
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
log =  LogisticRegression(C=1e10,max_iter=1000) 
log.fit(x_train, y_train)
preds = log.predict(x_test)
num = 0
preds = preds.tolist()
for i,pred in enumerate(preds):
    if int(pred) == int(y_test[i]):
        num += 1
print ('precision_score:' + str(float(num) / len(preds)))


*Logisticre Gression：
precision_score:0.8998120742306789


In [30]:
print ('*Random Forest Classifier：')
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
rfc =  RandomForestClassifier(n_estimators=500,  max_features='sqrt',random_state=10)
rfc.fit(x_train, y_train)
preds = log.predict(x_test)
num = 0
preds = preds.tolist()
for i,pred in enumerate(preds):
    if int(pred) == int(y_test[i]):
        num += 1
print ('precision_score:' + str(float(num) / len(preds)))

*Random Forest Classifier：
precision_score:0.8998120742306789


In [30]:
print ('*Decision Tree Classifier：')
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
dtc =  DecisionTreeClassifier(random_state=77)
dtc.fit(x_train, y_train)
preds = log.predict(x_test)
num = 0
preds = preds.tolist()
for i,pred in enumerate(preds):
    if int(pred) == int(y_test[i]):
        num += 1
print ('precision_score:' + str(float(num) / len(preds)))

*Decision Tree Classifier：


NameError: name 'log' is not defined